In [1]:
from pprint import pprint as pp

import pandas as pd
import sqlalchemy as db

### Connect

In [16]:
engine = db.create_engine('sqlite:///../hacking-pandas/census.sqlite')
connection = engine.connect()
metadata = db.MetaData()
census = db.Table('census', metadata, autoload=True, autoload_with=engine)

### Reflection

In [18]:
metadata.__dict__

{'tables': immutabledict({'census': Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)}),
 'schema': None,
 'naming_convention': immutabledict({'ix': 'ix_%(column_0_label)s'}),
 '_schemas': set(),
 '_sequences': {},
 '_fk_memos': defaultdict(list, {}),
 '_bind': None}

In [17]:
# Print the column names
print(census.columns.keys())

['state', 'sex', 'age', 'pop2000', 'pop2008']


In [4]:
#Equivalent to 'SELECT * FROM census'
query = db.select([census]) 

In [5]:
ResultProxy = connection.execute(query)

In [6]:
ResultSet = ResultProxy.fetchall()

In [7]:
df = pd.DataFrame(ResultSet)
df.columns=ResultSet[0].keys()
df

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547
3,Illinois,M,3,88868,90037
4,Illinois,M,4,91947,91111
...,...,...,...,...,...
8767,Texas,F,81,35378,44418
8768,Texas,F,82,33852,41838
8769,Texas,F,83,30076,40489
8770,Texas,F,84,27961,36821


In [8]:
q = db.select([census]).order_by(db.desc(census.columns.state), census.columns.pop2000)
ResultProxy = connection.execute(q)
ResultSet = ResultProxy.fetchall()
df = pd.DataFrame(ResultSet)
df.columns=ResultSet[0].keys()
df

,state,sex,age,pop2000,pop2008
0,Wyoming,M,84,490,643
1,Wyoming,M,83,515,726
2,Wyoming,M,82,634,792
3,Wyoming,M,81,687,845
4,Wyoming,F,84,801,878
...,...,...,...,...,...
8767,Alabama,F,41,35500,30240
8768,Alabama,F,42,35663,31693
8769,Alabama,F,40,35758,29999
8770,Alabama,F,43,35821,34069


In [9]:
# Let's play w/ functions
def _to_dataframe(cnx, query):
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    df = pd.DataFrame(ResultSet)
    df.columns=ResultSet[0].keys()
    return df

# Simple SELECT w/ ORDER BY
q = db.select([census]).order_by(db.desc(census.columns.state), census.columns.pop2000)
df = _to_dataframe(connection, q)
df

,state,sex,age,pop2000,pop2008
0,Wyoming,M,84,490,643
1,Wyoming,M,83,515,726
2,Wyoming,M,82,634,792
3,Wyoming,M,81,687,845
4,Wyoming,F,84,801,878
...,...,...,...,...,...
8767,Alabama,F,41,35500,30240
8768,Alabama,F,42,35663,31693
8769,Alabama,F,40,35758,29999
8770,Alabama,F,43,35821,34069


In [10]:
# Play w/ functions

# Sum - meh
q = db.select([db.func.sum(census.columns.pop2008).label('pop2008__total')])
df = _to_dataframe(connection, q)
df

,pop2008__total
0,302876613


In [11]:
# GROUP BY 
q = db.select(
    [
        db.func.sum(census.columns.pop2008).label('pop2008'), 
        census.columns.sex
    ]
).group_by(census.columns.sex)
_to_dataframe(connection, q)

,pop2008,sex
0,153959198,F
1,148917415,M


### Casts and cases

In [21]:
female_pop = db.func.sum(db.case([(census.columns.sex == 'F', census.columns.pop2000)],else_=0))
total_pop = db.cast(db.func.sum(census.columns.pop2000), db.Float)
query = db.select([female_pop/total_pop * 100])
result = connection.execute(query).scalar()
print(result)

51.09467432293413


In [22]:
engine = db.create_engine('sqlite:///census.sqlite')
connection = engine.connect()
metadata = db.MetaData()

census = db.Table('census', metadata, autoload=True, autoload_with=engine)
state_fact = db.Table('state_fact', metadata, autoload=True, autoload_with=engine)
query = db.select([census.columns.pop2008, state_fact.columns.abbreviation])
result = connection.execute(query).fetchall()
df = _to_dataframe(connection, query)
df

,pop2008,abbreviation
0,95012,IL
1,95012,NJ
2,95012,ND
3,95012,OR
4,95012,DC
...,...,...
447367,223439,AL
447368,223439,MN
447369,223439,NY
447370,223439,UT


In [25]:
query = db.select([state_fact])
df = _to_dataframe(connection, query)
df.head(3).transpose()

,0,1,2
id,13,30,34
name,Illinois,New Jersey,North Dakota
abbreviation,IL,NJ,ND
country,USA,USA,USA
type,state,state,state
sort,10,10,10
status,current,current,current
occupied,occupied,occupied,occupied
notes,,,
fips_state,17,34,38


#### Manual JOINs

In [26]:
query = db.select([census, state_fact])
query = query.select_from(census.join(state_fact, census.columns.state == state_fact.columns.name))
df = _to_dataframe(connection, query)
df.head()

,state,sex,age,pop2000,pop2008,id,name,abbreviation,country,type,...,occupied,notes,fips_state,assoc_press,standard_federal_region,census_region,census_region_name,census_division,census_division_name,circuit_court
0,Illinois,M,0,89600,95012,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
1,Illinois,M,1,88445,91829,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
2,Illinois,M,2,88729,89547,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
3,Illinois,M,3,88868,90037,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
4,Illinois,M,4,91947,91111,13,Illinois,IL,USA,state,...,occupied,,17,Ill.,V,2,Midwest,3,East North Central,7
